<a href="https://colab.research.google.com/github/brucenelm/homoyestageC/blob/master/StageCQuiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Importing data
df = pd.read_csv('https://raw.githubusercontent.com/rahatUlAin/Electrical-Grid-Stability-Simulated-Data-/master/Data_for_UCI_named.csv')

In [ ]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [ ]:
df.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4', 'stab', 'stabf'],
      dtype='object')

In [ ]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [ ]:
#Converting stabf into 1 and 0
df = pd.get_dummies(df)

In [ ]:
df.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4', 'stab', 'stabf_stable', 'stabf_unstable'],
      dtype='object')

In [ ]:
 #Setting up the independent and dependent variable
 X = df.drop(columns=['stab','stabf_stable', 'stabf_unstable'],axis=1)
 y = df['stabf_stable']

In [ ]:
X.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923


In [ ]:
y.head()

0    0
1    1
2    0
3    0
4    0
Name: stabf_stable, dtype: uint8

In [ ]:
#Split the data into training and testing
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 1)
#y_train.value_counts()

In [ ]:
#Using StandardScaler to standardise the the Indpedent data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)                
x_train = scaler.transform(x_train)
x_test  = scaler.transform(x_test) 

In [ ]:
#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=100, random_state=1)
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=100, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [ ]:
#RandomForestClassifier Accuracy
new_pred =clf.predict(x_test)
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, new_pred))


Accuracy: 0.9295


In [ ]:
#xgboost
from xgboost import XGBClassifier
xcl = XGBClassifier()
xcl.fit(x_train, y_train)
new_pred = xcl.predict(x_test)

pred=[]
for value in new_pred:
  pred.append(round(value))


In [ ]:
#xgboost Accuracy Calculated
accuracy = metrics.accuracy_score(y_test, pred)
print('Accuracy: {}'.format(round(accuracy*100),2))

Accuracy: 92.0


In [ ]:
#lightgbm
import lightgbm as ltb
lig = ltb.LGBMRegressor()
lig.fit(x_train, y_train)

predicted_y = lig.predict(x_test)
predicted_y= [round(value) for value in predicted_y]

In [ ]:
#lightgbm accuracy
accuracy = metrics.accuracy_score(y_test, predicted_y)
print('Accuracy: {}'.format(round(accuracy*100),2))

Accuracy: 93.0


In [ ]:
#Use scikit learn to train a random forest classifier
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state=1)
classifier.fit(x_train, y_train)

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]
hyperparameter_grid = {'n_estimators': n_estimators,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}
xtree = ExtraTreesClassifier()
rndsmodel = RandomizedSearchCV(xtree, hyperparameter_grid, random_state=1)
rndsmodel.fit(x_train, y_train)
xtree_pred = rndsmodel.predict(x_test)
xtree_pred

array([0, 0, 1, ..., 1, 0, 0], dtype=uint8)

In [77]:
accuracy = metrics.accuracy_score(y_test, xtree_pred)
print('Accuracy: {}'.format(round(accuracy*100),2))

Accuracy: 93.0


In [69]:
xtree.fit(x_train,y_train)
print(xtree.feature_importances_)

[0.11710766 0.11876434 0.11582033 0.1149472  0.03982005 0.04078351
 0.04026102 0.04020591 0.08824047 0.09391446 0.0969186  0.09321645]


In [72]:
X.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4'],
      dtype='object')

In [75]:
new_extra = ExtraTreesClassifier(random_state = 1)
new_extra.fit(x_train,y_train)
pred=new_extra.predict(x_test)

In [76]:
accuracy = metrics.accuracy_score(y_test, pred)
print('Accuracy: {}'.format(round(accuracy*100),2))

Accuracy: 93.0
